In [62]:
from bs4 import BeautifulSoup
import re
import requests

def get_soup(link):
    response = requests.get(link) 
    return BeautifulSoup(response.text, 'html.parser')

def get_string(element):
    print(element.string)
    try:
        return element.string
    except:
        return None

def get_selector_array(wrapper, selector):
    try:
        list_elements = list(wrapper.select(selector))
        items = []
        for element in list_elements:
            items.append(element.string)
        return items
    except:
        return []


def get_selector_string(wrapper, selector):
    try:
        return wrapper.select_one(selector).string
    except:
        return None


def parse_film(link): 
    soup = get_soup(link)
    film = {} 

    wrapper = soup.find('td', {'class': 'post'}) 

    film['url'] = link
    film['name_english'] = get_selector_string(wrapper, '.sub_title')
    film['name'] = get_selector_string(wrapper, 'h1.title')
    film['director'] = get_selector_string(wrapper, '.movie_info + p')
    film['rating'] = get_selector_string(wrapper, '.rating-big__value')
    film['year'] = get_selector_string(wrapper, '.year a')
    film['genres'] = get_selector_array(wrapper, '.genre p') 
    film['countries'] = get_selector_string(wrapper, '.author') 
    film['duration'] = get_selector_string(wrapper, '.duration') 

    return film


def print_film(film):
    print('URL:', film['url'])
    print('{} ({}), {} | {}'.format(film['name'], film['name_english'], film['year'], film['duration']))
    print('Рейтинг: {}'.format(film['rating']))
    print('Страны: {}'.format(film['countries']))
    print('Жанры: {}'.format(', '.join(film['genres'])))


films_link = 'https://afisha.tut.by/film/'
response = requests.get(films_link) 
soup = BeautifulSoup(response.text, 'html.parser')
films = soup.findAll('li', {'class': 'lists__li'}) 
links = set() 

for listitem in films: 
    link_element = listitem.find('a', {'class': 'name'})
    if link_element is not None: 
        link = link_element['href']
        try:
            short_link = re.findall(r'https:\/\/afisha.tut.by\/film\/.*\/', link)[0]
        except:
            short_link = None
        if short_link is not None: 
            links.add(short_link)

results = {}

for link in links: 
    try:
        film = parse_film(link)
        print_film(film)
        print('\n')
    except Exception as e:
        print(e)

 

URL: https://afisha.tut.by/film/odnazhdy-tarantino/
Однажды… Тарантино (None), None | 120 минут
Рейтинг: None
Страны: США
Жанры: Документальный


URL: https://afisha.tut.by/film/porko_rosso/
Порко Россо (Porco Rosso), None | None
Рейтинг: 7,2
Страны: None
Жанры: 


URL: https://afisha.tut.by/film/temnye-vody/
Темные воды (Премьера), 2019 | 126 минут
Рейтинг: None
Страны: США
Жанры: 


URL: https://afisha.tut.by/film/otverzhennye-1/
Отверженные (None), 2019 | 102 минуты
Рейтинг: 7,2
Страны: Франция
Жанры: Драма


URL: https://afisha.tut.by/film/gurvinek-volshebnaya-igra/
Гурвинек. Волшебная игра (6+), 2017 | 80 минут
Рейтинг: 3,7
Страны:  Россия, Чехия, Бельгия 
Жанры: Детский/Семейный


URL: https://afisha.tut.by/film/oldeuboi/
Олдбой (18+), None | 120 минут
Рейтинг: 8,2
Страны: Южная Корея
Жанры: Боевик, Детектив, Драма, Триллер


URL: https://afisha.tut.by/film/mmkf-listapad-pravda/
Правда (Премьера), 2019 | 106 минут
Рейтинг: 6,8
Страны: Франция, Япония
Жанры: Драма


URL: https://a